In [37]:
!git config --global user.email "naazshafeer@gmail.com"
!git config --global user.name "Naaz Shafeer"

This will be used to sort the Liu data. Our aim is to get slightly upwards of 1280 pairs

In [22]:
import pandas as pd
import os

file = os.path.join("data", "data/Dataframe_csv", "data_sdss_agn_minimized.csv")

df = pd.read_csv(file)
df.head()

,sdss designation,redshift,angular sep,proper sep,sdss r band
0,J000249.07+004504.8,0.0868,5.9,9.5,16.08
1,J000249.44+004506.7,0.0865,5.9,9.5,16.43
2,J000257.21+000750.5,0.0901,45.4,76.0,16.15
3,J000258.59+000831.0,0.0897,45.4,76.0,16.67
4,J000323.74+010547.3,0.0993,12.2,22.4,17.98


Sorting algorithm that will take the OG csv and then will sort and then flag the ones that are unsorted and then make a copy of the dataframe and then its a forloop until there arent anymore grouped pairs or systems. This will use parameters of proper separation for the first trial, and then proper separation, hop and then compare redshifts and then do 5 more hops and if it doesnt find a match then that system is closed (for that redshift). Goal is upwards of 1280

In [23]:
df_file = pd.DataFrame(df) #just doing this cuz this is how we do it in class
df_file.head()

,sdss designation,redshift,angular sep,proper sep,sdss r band
0,J000249.07+004504.8,0.0868,5.9,9.5,16.08
1,J000249.44+004506.7,0.0865,5.9,9.5,16.43
2,J000257.21+000750.5,0.0901,45.4,76.0,16.15
3,J000258.59+000831.0,0.0897,45.4,76.0,16.67
4,J000323.74+010547.3,0.0993,12.2,22.4,17.98


In [33]:
def first_sort(filename):
    id = 1
    flag = [id]
    
    for i in range(1, len(filename)):
        matched = False
        for j in range(max(0, i-5), i):
            if abs(filename["proper sep"].iloc[i] - filename["proper sep"].iloc[j]) < 0.001:
                flag.append(flag[j])
                matched = True
                break
        if not matched: #so basically if matched
            id += 1
            flag.append(id)
    return flag #will output a list which we can append as a new column in our dataframe



def sec_sort(filename):
    id = 1
    flag = [id]
    
    for i in range(1, len(filename)):
        matched = False
        for j in range(max(0, i-3), i):
            if abs(filename["proper sep"].iloc[i] - filename["proper sep"].iloc[j]) < 0.001 and abs(filename["redshift"].iloc[i] - filename["redshift"].iloc[j]) < 0.02:
                flag.append(flag[j])
                matched = True
                break
        if not matched: #so basically if matched
            id += 1
            flag.append(id)
    return flag #will output a list which we can append as a new column in our dataframe


# def sec_sort(filename):
#     flag = []
#     count = 0
#     for val in filename["redshift"]:
#         if val == val + 1: #the row and then next row but need to add tolerance of 0.001 aka if they are matching in e-3 then they are in the same group
#             flag.append(count)
#         elif val != val + 1:
#             flag.append("Unmatched")
#         count += 1
#     return flag #will output a list which we can append as a new column in our dataframe

# def final_sort(filename):
#     flag = []
#     count = 0
#     for val in filename["morphology"]:
#         if val == val + 1: #the row and then next row
#             flag.append(count)
#         elif val != val + 1:
#             flag.append("Unmatched")
#         count += 1
#     return flag #will output a list which we can append as a new column in our dataframe





Now let's see and do this sort to see if flagged correctly

In [34]:
first_sort_file = df_file.copy()
first_sort_trial = first_sort(first_sort_file)
df_file["Groupings"] = first_sort_trial

df_file.head() #so first sort worked???

,sdss designation,redshift,angular sep,proper sep,sdss r band,Groupings
0,J000249.07+004504.8,0.0868,5.9,9.5,16.08,1
1,J000249.44+004506.7,0.0865,5.9,9.5,16.43,1
2,J000257.21+000750.5,0.0901,45.4,76.0,16.15,2
3,J000258.59+000831.0,0.0897,45.4,76.0,16.67,2
4,J000323.74+010547.3,0.0993,12.2,22.4,17.98,3


In [35]:
sec_sort_file = df_file.copy()

second_sort_trial = sec_sort(sec_sort_file)
df_file["Groupings"] = second_sort_trial

df_file.head() #so second sort worked???

,sdss designation,redshift,angular sep,proper sep,sdss r band,Groupings
0,J000249.07+004504.8,0.0868,5.9,9.5,16.08,1
1,J000249.44+004506.7,0.0865,5.9,9.5,16.43,1
2,J000257.21+000750.5,0.0901,45.4,76.0,16.15,2
3,J000258.59+000831.0,0.0897,45.4,76.0,16.67,2
4,J000323.74+010547.3,0.0993,12.2,22.4,17.98,3


In [36]:
flag_count = df_file.groupby("Groupings").size()

print("Single : {}".format(sum(flag_count == 1)))
print("Dual : {}".format(sum(flag_count == 2)))
print("Trinary : {}".format(sum(flag_count == 3)))
print("Quad: {}".format(sum(flag_count == 4)))
print("Quinuple : {}".format(sum(flag_count == 5)))

Single : 76
Dual : 1269
Trinary : 0
Quad: 1
Quinuple : 0


In [38]:
flag_count = first_sort_file.groupby("Groupings").size()

print("Single : {}".format(sum(flag_count == 1)))
print("Dual : {}".format(sum(flag_count == 2)))
print("Trinary : {}".format(sum(flag_count == 3)))
print("Quad: {}".format(sum(flag_count == 4)))
print("Quinuple : {}".format(sum(flag_count == 5)))

Single : 52
Dual : 1281
Trinary : 0
Quad: 1
Quinuple : 0


In [39]:
flag_count = sec_sort_file.groupby("Groupings").size()

print("Single : {}".format(sum(flag_count == 1)))
print("Dual : {}".format(sum(flag_count == 2)))
print("Trinary : {}".format(sum(flag_count == 3)))
print("Quad: {}".format(sum(flag_count == 4)))
print("Quinuple : {}".format(sum(flag_count == 5)))

Single : 52
Dual : 1275
Trinary : 0
Quad: 4
Quinuple : 0


Do a sanity check
1. Once you do groupby gorupings - check if within one of these groups are the seprations equal, so 1275 pairs should have equal separation?
2. If they do have equal separations - is their redshift relatively equal?
3. You have pairs - from the pairs - you can use WISE to get bol lumn to get black hole mass. Wise cannot resolve pairs under blan resolution - for most of these pairs you migtht just get one measurement so you will get to the total black hole mass. So you can make some assumtpions (like equal mass). Go by the flux and get hte flux ratio - from r band mag. 

Black hole mass is one thing that we wanted. 

Which pair has two measurements from WISE and which pairss have one measurement from WISE - if only one measurement and the separation than 20 kpc. If above 20 kpc and it has only one measurement. 

Plate scale is how large one angle and pixel relate

One pixel, 2.75 arcsec - WISE
SDSS 0.4 arcsec
turbulence in atmosphere - seeing will be larger. seeing will determine if we can resolve a target or not

Since, the plate scale about 3 arcsec, so minimum resolvution is 6 arcsec - so what I could do is 6 arcsec and convert it and use redshift and angular stuff and convert 

Consistency check  - using angular sep and plate scale to be able to backcheck. convert arcsec to physicsla distances to kps - compare the seeing (2.75 * 2-3) to the angular sep the Liu data give me. If the angular separation is 2 arcsec. Once you match it and then get the mass - tbale of duals and rhe mass and the separation. something else for galaxy stuff???



Null - dont throw away null lines 